In [1]:
import numpy as np
import models
import detect
import csv
from tqdm import tqdm
from rouge_score import rouge_scorer
import pickle

2024-11-26 04:27:51.869049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 04:27:52.434124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/jamir23/.cache/huggingface/token
Login successful


In [2]:
models.set_seed(32)
models.initialize_models()

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.


In [5]:
main_question = input("Main Question: ").strip()
final_response = detect.internally_validated_response(main_question, val_threshold=0.1)
#final_response, _ = detect.externally_validated_response("4cece08808794206b4a8fb6770e51633", main_question, val_threshold=0.1)
#final_response, _ = detect.agent_validated_response(models.agent_model, main_question, val_threshold=0.1)

Main Question:  Why have vaccines been fatal during the pandemic?


Uncertain Claim: Vaccines have been fatal due to the severity of the COVID-19 disease.

Uncertain Proportion Score: 0.12

Validating...

╭──────────────────────────────────────────────── Validated Claim ────────────────────────────────────────────────╮
│ Validation Result: The claim that vaccines have been fatal during the pandemic is not accurate. Vaccines have   │
│ been used to prevent and treat COVID-19, and they have been shown to be safe and effective in clinical trials.  │
│ The severity of COVID-19 is a complex factor that cannot be attributed to the vaccines alone.                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

The Max Uncertainty For Current Response Is: 0.19

Generating Response With Internally Validated Data...

The Max Uncertainty For Current Response Is: 0.04

╭──────────────────────────────────────────────── Final Response ─────────────────────────────────────────────────╮
│ {                                                                                                               │
│  "question": "Why have vaccines been fatal during the pandemic?",                                               │
│  "reference": [                                                                                                 │
│   "The claim that vaccines have been fatal during the pandemic is not accurate. Vaccines have been used to      │
│ prevent and treat COVID-19, and they have been shown to be safe and effective in clinical trials. The severity  │
│ of COVID-19 is a complex factor that cannot be attributed to the vaccines alone.",                              │
│   "Vaccines have also been known to cause blood clotting disorders and autoimmune reactions. However, the claim │
│ that vaccines have been fatal during the pandemic is incorrect."                                                │
│  ],                                                                                                             │
│  "response": "Vaccines have been used to prevent and treat COVID-19, and they have been shown to be safe and    │
│ effective in clinical trials. The severity of COVID-19 is a complex factor that cannot be attributed to the     │
│ vaccines alone.",                                                                                               │
│  "main_points": [                                                                                               │
│   "Vaccines have been used to prevent and treat COVID-19, and they have been shown to be safe and effective in  │
│ clinical trials.",                                                                                              │
│   "The severity of COVID-19 is a complex factor that cannot be attributed to the vaccines alone."               │
│  ]                                                                                                              │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Internally Validated Answer ──────────────────────────────────────────╮
│ Vaccines have been used to prevent and treat COVID-19, and they have been shown to be safe and effective in     │
│ clinical trials. The severity of COVID-19 is a complex factor that cannot be attributed to the vaccines alone.  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
questions = []
correct_answers = []
incorrect_answers = []

file_path = 'TruthfulQA.csv'

with open(file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        questions.append(row['Question'])
        correct_answers.append([ans.strip() for ans in row['Correct Answers'].split(';')])
        incorrect_answers.append([f"{ans.strip()}. True" for ans in row['Incorrect Answers'].split(';')])

questions = questions[:100]

In [5]:
def calculate_scores(final_response, correct_answers, judge):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    rouge_score = 0
    judge_score = 0
    num_answers = len(correct_answers)

    for correct_answer in correct_answers:
        temp_judge_score = judge.predict([(final_response, correct_answer)]).cpu().item()
            
        if temp_judge_score > judge_score:
            judge_score = temp_judge_score
            
            rouge_scores = scorer.score(final_response, correct_answer)
            #print((correct_answer, final_response))
            rouge_score = (rouge_scores['rouge1'].fmeasure + rouge_scores['rouge2'].fmeasure + rouge_scores['rougeL'].fmeasure) / 3
            
    return rouge_score, judge_score

In [ ]:
total_questions = len(questions)
initial_responses = []

In [ ]:
total_rouge = 0
correct_judged = 0

for i, question in tqdm(enumerate(questions), total=total_questions, desc="Evaluating Responses"):
    final_response, initial_response = detect.internally_validated_response(question, visualize=False, val_threshold=0.1)
    initial_responses.append(initial_response)
    
    rouge_score, judge_score = calculate_scores(final_response, correct_answers[i], models.agent_model)
    total_rouge += rouge_score

    if judge_score > 0.5:
        correct_judged += 1

average_rouge = total_rouge / total_questions
correct_percentage = (correct_judged / total_questions) * 100

print("INTERNAL VALIDATION:")
print(f"Average ROUGE: {average_rouge:.4f}")
print(f"Correctly Answered: {correct_percentage:.2f}%")

Evaluating Responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:52<00:00,  4.72s/it]

INTERNAL VALIDATION:
Average ROUGE: 0.2177
Correctly Answered: 46.00%


In [7]:
total_rouge = 0
correct_judged = 0

for i, question in tqdm(enumerate(questions), total=total_questions, desc="Evaluating Responses"):
    final_response, _ = detect.externally_validated_response("4cece08808794206b4a8fb6770e51633", question, visualize=False, val_threshold=0.1, initial_response=initial_responses[i])
    rouge_score, judge_score = calculate_scores(final_response, correct_answers[i], models.agent_model)
    total_rouge += rouge_score

    if judge_score > 0.5:
        correct_judged += 1

average_rouge = total_rouge / total_questions
correct_percentage = (correct_judged / total_questions) * 100

print("EXTERNAL VALIDATION:")
print(f"Average ROUGE: {average_rouge:.4f}")
print(f"Correctly Answered: {correct_percentage:.2f}%")

Evaluating Responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [09:26<00:00,  5.66s/it]

EXTERNAL VALIDATION:
Average ROUGE: 0.2050
Correctly Answered: 49.00%


In [9]:
total_rouge = 0
correct_judged = 0

for i, question in tqdm(enumerate(questions), total=total_questions, desc="Evaluating Responses"):
    final_response, _ = detect.agent_validated_response(models.agent_model, question, visualize=False, val_threshold=0.1, initial_response=initial_responses[i])
    rouge_score, judge_score = calculate_scores(final_response, correct_answers[i], models.agent_model)
    total_rouge += rouge_score

    if judge_score > 0.5:
        correct_judged += 1

average_rouge = total_rouge / total_questions
correct_percentage = (correct_judged / total_questions) * 100

print("AGENT VALIDATION:")
print(f"Average ROUGE: {average_rouge:.4f}")
print(f"Correctly Answered: {correct_percentage:.2f}%")

Evaluating Responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:44<00:00,  1.64s/it]

AGENT VALIDATION:
Average ROUGE: 0.2419
Correctly Answered: 48.00%


In [10]:
total_questions = len(questions)

total_rouge = 0
correct_judged = 0

for i, question in tqdm(enumerate(questions), total=total_questions, desc="Evaluating Responses"):
    final_response, _ = detect.internally_validated_response(question, visualize=False, val_threshold=10, initial_response=initial_responses[i])
    rouge_score, judge_score = calculate_scores(final_response, correct_answers[i], models.agent_model)
    total_rouge += rouge_score

    if judge_score > 0.5:
        correct_judged += 1

average_rouge = total_rouge / total_questions
correct_percentage = (correct_judged / total_questions) * 100

print("NO VALIDATION:")
print(f"Average ROUGE: {average_rouge:.4f}")
print(f"Correctly Answered: {correct_percentage:.2f}%")

Evaluating Responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.57it/s]

NO VALIDATION:
Average ROUGE: 0.2416
Correctly Answered: 44.00%
